<a href="https://colab.research.google.com/github/ashishpal2702/HumanActivityrecognition/blob/main/Logistic_Regression_and_Classification_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

We will be using the [Human Activity Recognition with Smartphones](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) database, which was built from the recordings of study participants performing activities of daily living (ADL) while carrying a smartphone with an embedded inertial sensors. The objective is to classify activities into one of the six activities (walking, walking upstairs, walking downstairs, sitting, standing, and laying) performed.

For each record in the dataset it is provided: 

- Triaxial acceleration from the accelerometer (total acceleration) and the estimated body acceleration. 
- Triaxial Angular velocity from the gyroscope. 
- A 561-feature vector with time and frequency domain variables. 
- Its activity label. 

More information about the features is available on the website above.

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [42]:
from __future__ import print_function
import os
data_path = [ 'data']

from sklearn.preprocessing import LabelEncoder , StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import ExtraTreesClassifier

In [9]:
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline

## 1. Data Import

Import the data and do the following:

* Examine the data types--there are many columns, so it might be wise to use value counts
* Determine if the floating point values need to be scaled
* Determine the breakdown of each activity
* Encode the activity label as an integer

In [45]:
import pandas as pd
import numpy as np
import os
#filepath = '/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/data//Human_Activity_Recognition_Using_Smartphones_Data_augmented_data.gzip'
filepath = '/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/data/Human_Activity_Recognition_Using_Smartphones_Data_augmented_timedata_sample.gzip'
data = pd.read_parquet(filepath)

In [11]:
data.shape

(100000, 563)

In [12]:
#data = data.sample(10000)

In [13]:
sensors = set()
for col in data.columns:
  sensors.add(col.split("-")[0])

In [14]:
sensors

{'Activity',
 'angle(X,gravityMean)',
 'angle(Y,gravityMean)',
 'angle(Z,gravityMean)',
 'angle(tBodyAccJerkMean),gravityMean)',
 'angle(tBodyAccMean,gravity)',
 'angle(tBodyGyroJerkMean,gravityMean)',
 'angle(tBodyGyroMean,gravityMean)',
 'date_time',
 'fBodyAcc',
 'fBodyAccJerk',
 'fBodyAccMag',
 'fBodyBodyAccJerkMag',
 'fBodyBodyGyroJerkMag',
 'fBodyBodyGyroMag',
 'fBodyGyro',
 'tBodyAcc',
 'tBodyAccJerk',
 'tBodyAccJerkMag',
 'tBodyAccMag',
 'tBodyGyro',
 'tBodyGyroJerk',
 'tBodyGyroJerkMag',
 'tBodyGyroMag',
 'tGravityAcc',
 'tGravityAccMag'}

In [15]:
data['Activity'].value_counts()

Activity
LAYING                16762
WALKING               16728
WALKING_UPSTAIRS      16675
STANDING              16645
WALKING_DOWNSTAIRS    16627
SITTING               16563
Name: count, dtype: int64

In [16]:
data.describe().T

,count,mean,min,25%,50%,75%,max,std
tBodyAcc-mean()-X,100000.0,0.275078,-0.986314,0.262825,0.277178,0.288289,0.965808,0.060068
tBodyAcc-mean()-Y,100000.0,-0.017384,-0.89446,-0.024411,-0.017155,-0.011201,0.87536,0.029089
tBodyAcc-mean()-Z,100000.0,-0.108674,-0.936031,-0.120134,-0.108572,-0.098326,0.999455,0.044118
tBodyAcc-std()-X,100000.0,-0.564353,-1.0,-0.991627,-0.620961,-0.211144,1.0,0.450584
tBodyAcc-std()-Y,100000.0,-0.467669,-0.999629,-0.974517,-0.382965,-0.028442,0.999009,0.503729
...,...,...,...,...,...,...,...,...
"angle(tBodyGyroJerkMean,gravityMean)",100000.0,-0.013572,-1.0,-0.339358,-0.012546,0.30096,0.988742,0.426647
"angle(X,gravityMean)",100000.0,-0.523201,-0.999923,-0.816508,-0.724015,-0.554554,0.997924,0.487325
"angle(Y,gravityMean)",100000.0,0.079965,-0.997831,0.029963,0.18861,0.253884,0.999779,0.293338
"angle(Z,gravityMean)",100000.0,-0.041572,-0.99383,-0.103348,0.002657,0.109922,0.999776,0.256959


In [17]:
le = LabelEncoder()
data['Activity'] = le.fit_transform(data['Activity'])

In [18]:
data['Activity'].value_counts()

Activity
0    16762
3    16728
5    16675
2    16645
4    16627
1    16563
Name: count, dtype: int64

## 2. EDA
 
* Calculate the correlations between the dependent variables.
* Identify high correlated features and drop them.

In [19]:
data_features = data.drop('Activity' , axis = 1)
# Create correlation matrix
corr_matrix = data_features.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.80
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
print(to_drop)

['tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z', 'tBodyAcc-max()-X', 'tBodyAcc-max()-Y', 'tBodyAcc-max()-Z', 'tBodyAcc-min()-X', 'tBodyAcc-min()-Y', 'tBodyAcc-min()-Z', 'tBodyAcc-sma()', 'tBodyAcc-energy()-X', 'tBodyAcc-energy()-Y', 'tBodyAcc-energy()-Z', 'tBodyAcc-iqr()-X', 'tBodyAcc-iqr()-Y', 'tBodyAcc-iqr()-Z', 'tBodyAcc-entropy()-X', 'tBodyAcc-entropy()-Y', 'tBodyAcc-entropy()-Z', 'tBodyAcc-arCoeff()-X,1', 'tBodyAcc-arCoeff()-X,2', 'tBodyAcc-arCoeff()-X,3', 'tBodyAcc-arCoeff()-Y,2', 'tBodyAcc-arCoeff()-Z,2', 'tGravityAcc-mad()-X', 'tGravityAcc-mad()-Y', 'tGravityAcc-mad()-Z', 'tGravityAcc-max()-X', 'tGravityAcc-max()-Y', 'tGravityAcc-max()-Z', 'tGravityAcc-min()-X', 'tGravityAcc-min()-Y', 'tGravityAcc-min()-Z', 'tGravityAcc-energy()-X', 'tGravityAcc-energy()-Y', 'tGravityAcc-energy()-Z', 'tGravityAcc-iqr()-X', 'tGravityAcc-iqr()-Y', 'tGravityAcc-iqr()-Z', 'tGravityAcc-arCoeff()-X,2', 'tGravityAcc-arCoeff()-X,3', 'tGravityAcc-arCoe

In [20]:
len(to_drop)

420

In [21]:
# Drop features
data.drop(to_drop, axis=1, inplace=True)

### Feature engineering

In [36]:
from sklearn.model_selection import train_test_split
X = data.drop(['date_time','Activity'] , axis = 1)
Y = data['Activity']

In [43]:
def get_top_k_features(X, Y, k):
    clf = ExtraTreesClassifier(n_estimators=50)
    clf = clf.fit(X, Y)
    feature_df = pd.DataFrame(data=(X.columns, clf.feature_importances_)).T.sort_values(by=1, ascending=False)
    cols = feature_df.head(k)[0].values
    return cols

In [44]:
top_k_features = get_top_k_features(X, Y, k=10)

In [45]:
top_k_features

array(['tGravityAcc-mean()-X', 'tGravityAcc-mean()-Y', 'tBodyAcc-std()-X',
       'tGravityAcc-mean()-Z', 'tBodyAcc-correlation()-X,Y',
       'tGravityAcc-arCoeff()-Z,1', 'tBodyGyro-entropy()-Z',
       'tGravityAcc-arCoeff()-X,1', 'tGravityAcc-sma()',
       'tGravityAcc-arCoeff()-Y,1'], dtype=object)

## 3. Data preparation

* Split the data into train and test data sets. 
* Regardless of methods used to split the data, compare the ratio of classes in both the train and test splits.


In [46]:
X = X[top_k_features]

In [47]:
x_train , x_test , y_train , y_test = train_test_split(X, Y)

In [48]:
x_train.shape , y_train.shape

((75000, 10), (75000,))

In [49]:
x_test.shape , y_test.shape

((25000, 10), (25000,))

## Standardise data

In [50]:
sc = StandardScaler()
x_train_std = sc.fit_transform(x_train)
x_test_std = sc.transform(x_test)

## 4. Model Training

* Fit different models and compare the result 
1. Logistic regression
2. Decision Tree Classifier
3. Random Forest Classifier 
4. Adaptive Boosting Classifier

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier


In [52]:
model_result = {}

In [53]:
lr =  LogisticRegression()
lr.fit(x_train_std, y_train)
train_accuracy = round(lr.score(x_train_std, y_train)*100,2)
test_accuracy = round(lr.score(x_test_std, y_test)*100,2)
print("Training Accuracy", train_accuracy)
print("Test Accuracy", test_accuracy)
model_result['Logistic_Regression'] = {'train_accuracy': train_accuracy,'test_accuracy':test_accuracy }

Training Accuracy 93.36
Test Accuracy 93.34


/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/code/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [54]:
dt =  DecisionTreeClassifier()
dt.fit(x_train, y_train)
train_accuracy = round(dt.score(x_train, y_train)*100,2)
test_accuracy = round(dt.score(x_test, y_test)*100,2)
print("Training Accuracy", train_accuracy)
print("Test Accuracy", test_accuracy)
model_result['Decision_Tree'] = {'train_accuracy': train_accuracy,'test_accuracy':test_accuracy }

Training Accuracy 100.0
Test Accuracy 98.59


In [55]:
rfc =  RandomForestClassifier()
rfc.fit(x_train, y_train)
train_accuracy = round(rfc.score(x_train, y_train)*100,2)
test_accuracy = round(rfc.score(x_test, y_test)*100,2)
print("Training Accuracy", train_accuracy)
print("Test Accuracy", test_accuracy)
model_result['RandomForest'] = {'train_accuracy': train_accuracy,'test_accuracy':test_accuracy }

Training Accuracy 100.0
Test Accuracy 99.59


In [56]:
abc =  AdaBoostClassifier()
abc.fit(x_train, y_train)
train_accuracy = round(abc.score(x_train, y_train)*100,2)
test_accuracy = round(abc.score(x_test, y_test)*100,2)
print("Training Accuracy", train_accuracy)
print("Test Accuracy", test_accuracy)
model_result['AdaBoost'] = {'train_accuracy': train_accuracy,'test_accuracy':test_accuracy }

Training Accuracy 49.97
Test Accuracy 49.28


## 5. Model comparison

In [57]:
pd.DataFrame(model_result).T

,train_accuracy,test_accuracy
Logistic_Regression,93.36,93.34
Decision_Tree,100.00,98.59
RandomForest,100.00,99.59
AdaBoost,49.97,49.28


### 6. Final Model

In [58]:
importances = rfc.feature_importances_


In [59]:
## Top 10 Features contributing to Model 
feature_df = pd.DataFrame( x_train.columns ,  columns = ['variables'])
feature_df['importance'] = importances
feature_df.sort_values( by = 'importance' , ascending = False).head(10)

,variables,importance
2,tBodyAcc-std()-X,0.226155
1,tGravityAcc-mean()-Y,0.176024
0,tGravityAcc-mean()-X,0.151211
4,"tBodyAcc-correlation()-X,Y",0.075225
3,tGravityAcc-mean()-Z,0.073888
9,"tGravityAcc-arCoeff()-Y,1",0.068945
6,tBodyGyro-entropy()-Z,0.067076
5,"tGravityAcc-arCoeff()-Z,1",0.066763
7,"tGravityAcc-arCoeff()-X,1",0.062455
8,tGravityAcc-sma(),0.032257


In [60]:
y_pred = rfc.predict(x_test)

## 7. Model Evaluation

For each model, calculate the following error metrics: 

* accuracy
* precision
* recall
* fscore
* confusion matrix

Decide how to combine the multi-class metrics into a single value for each model.

In [61]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report


In [62]:
confusion_matrix(y_pred, y_test)

array([[4221,    0,    0,    0,    0,    0],
       [   0, 4002,   50,    0,    0,    0],
       [   0,   47, 4152,    0,    0,    0],
       [   0,    0,    0, 4200,    0,    1],
       [   0,    0,    0,    1, 4084,    0],
       [   0,    0,    0,    3,    1, 4238]])

In [63]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4221
           1       0.99      0.99      0.99      4052
           2       0.99      0.99      0.99      4199
           3       1.00      1.00      1.00      4201
           4       1.00      1.00      1.00      4085
           5       1.00      1.00      1.00      4242

    accuracy                           1.00     25000
   macro avg       1.00      1.00      1.00     25000
weighted avg       1.00      1.00      1.00     25000



## 8. Model registration


In [65]:
## Change Drive path to your Folder
#os.mkdir('../model_weights/')
#os.mkdir('../model_features/')

In [ ]:
## Let's save model weights
import joblib
# save
joblib.dump(rfc, "../model_weights/my_random_forest.joblib")

In [ ]:
## Save Final columns
final_columns = np.array(x_train.columns) 
joblib.dump(final_columns, "../model_features/train_features.joblib")

## 9. Model Prediction

In [ ]:
## load Test Data set 
test_data = pd.read_csv('/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/data/Human_Activity_Recognition_Using_Smartphones_TestData.csv')

In [ ]:
## Load Features and model weight
train_features = joblib.load("../model_features/train_features.joblib")

model = joblib.load("../model_weights/my_random_forest.joblib")


In [ ]:
test_data_features = test_data[train_features]

In [ ]:
test_data_features.fillna(0,inplace = True)

In [ ]:
y_prediction = model.predict(test_data_features)
y_prediction_label = le.inverse_transform(y_prediction)
test_data['Prediction_label'] = y_prediction_label

In [ ]:
test_data.head()